Tutorial: https://towardsdatascience.com/implementing-an-autoencoder-in-tensorflow-2-0-5e86126e9f7
GitHub code: https://gist.github.com/AFAgarap/326af55e36be0529c507f1599f88c06e

In [2]:
import numpy as np
import tensorflow as tf  # TF 2.0 required
%load_ext tensorboard

In [10]:
np.random.seed(1)
tf.random.set_seed(1)

learning_rate = 1e-2
n_batches = 600
epochs = 10

# hidden features for compression
# We use 2d to be able to plot (or 3d would work too)
# TODO: compress less (e.g. 32 features), then use t-SNE to compress to 2d
compr_dim = 2

In [5]:
# Simple autoencoder with one encoding and one decoding layer.
class Encoder(tf.keras.layers.Layer):
  def __init__(self, compr_dim):
    super(Encoder, self).__init__()
    self.hidden_layer = tf.keras.layers.Dense(
      units=compr_dim,
      activation=tf.nn.relu,
      kernel_initializer='he_uniform'
    )
    self.output_layer = tf.keras.layers.Dense(
      units=compr_dim,
      activation=tf.nn.relu,
      kernel_initializer='he_uniform'
    )
    
  def call(self, input_features):
    return self.output_layer(self.hidden_layer(input_features))


class Decoder(tf.keras.layers.Layer):
  def __init__(self, compr_dim, original_dim):
    super(Decoder, self).__init__()
    self.hidden_layer = tf.keras.layers.Dense(
      units=compr_dim,
      activation=tf.nn.relu,
      kernel_initializer='he_uniform'
    )
    self.output_layer = tf.keras.layers.Dense(
      units=original_dim,
      activation=tf.nn.relu,
      kernel_initializer='he_uniform'
    )
  
  def call(self, code):
    return self.output_layer(self.hidden_layer(code))



class Autoencoder(tf.keras.Model):
  def __init__(self, compr_dim, original_dim):
    super(Autoencoder, self).__init__()
    self.encoder = Encoder(compr_dim=compr_dim)
    self.decoder = Decoder(compr_dim=compr_dim, original_dim=original_dim)
  
  def call(self, input_features):
    code = self.encoder(input_features)
    reconstructed = self.decoder(code)
    return reconstructed

In [6]:
def loss(model, original):
  reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model(original), original)))
  return reconstruction_error


# Training function with imperative forward pass.
def train(loss, model, opt, original):
  with tf.GradientTape() as tape:
    gradients = tape.gradient(loss(model, original), model.trainable_variables)
    gradient_variables = zip(gradients, model.trainable_variables)
    opt.apply_gradients(gradient_variables)

In [13]:
# Load the MNIST dataset.
# Training labels only used to identify clusters in 2d plot, not for classification (unsupervised learning).
(training_features, training_labels), (test_features, _) = tf.keras.datasets.mnist.load_data()
n_sample = training_features.shape[0]
dim_x = training_features.shape[1]
dim_y = training_features.shape[2]
original_dim = dim_x * dim_y
batch_size   = n_sample // n_batches

# Normalize and flatten data.
training_features = training_features / np.max(training_features)
training_features = training_features.reshape(n_sample, original_dim)
training_features = training_features.astype('float32')

# Convert to TF input pipeline, batch and shuffle.
training_dataset = tf.data.Dataset.from_tensor_slices(training_features)
training_dataset = training_dataset.batch(batch_size)
training_dataset = training_dataset.shuffle(n_sample)
training_dataset = training_dataset.prefetch(batch_size * 4)

In [11]:
# Summary file for TensorBoard
writer = tf.summary.create_file_writer('tb/autoencoder/run1')

# Instantiate autoencoder model and optimization function.
autoencoder = Autoencoder(compr_dim=compr_dim, original_dim=original_dim)
opt = tf.optimizers.Adam(learning_rate=learning_rate)

# Training loop
with writer.as_default():
  with tf.summary.record_if(True):
    for epoch in range(epochs):
      for step, batch_features in enumerate(training_dataset):
        train(loss, autoencoder, opt, batch_features)
        loss_values = loss(autoencoder, batch_features)
        original = tf.reshape(batch_features, (batch_features.shape[0], dim_x, dim_y, 1))
        reconstructed = tf.reshape(autoencoder(tf.constant(batch_features)), (batch_features.shape[0], dim_x, dim_y, 1))
        tf.summary.scalar('loss', loss_values, step=step)
        tf.summary.image('original', original, max_outputs=10, step=step)
        tf.summary.image('reconstructed', reconstructed, max_outputs=10, step=step)

In [15]:
# TODO: plot 2d compressed images, with label-based coloring (training_labels)
compressed = autoencoder.encoder(tf.constant(training_features))
# converto numpy array, then plot. or use tfplot:
# https://tensorflow-plot.readthedocs.io/en/latest/guide/showcases.html
# import tfplot

In [17]:
%tensorboard --logdir tb/autoencoder

Reusing TensorBoard on port 6006 (pid 4630), started 0:10:30 ago. (Use '!kill 4630' to kill it.)